<a href="https://colab.research.google.com/github/Ditipriya5678/Coronavirus-genome-sequence/blob/main/Coronavirus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install keras --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.11.0
    Uninstalling keras-2.11.0:
      Successfully uninstalled keras-2.11.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.11.0 requires keras<2.12,>=2.11.0, but you have keras 2.12.0 which is incompatible.


In [ ]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached keras-2.11.0-py2.py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0


In [ ]:
from Bio import SeqIO
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the genome sequences in FASTA format
sequences = []
labels = []
for record in SeqIO.parse("coronavirus.fasta", "fasta"):
    sequences.append(record.seq)
    labels.append(record.id)

# Calculate the GC content of each sequence
gc_content = np.array([100.0 * (seq.count("G") + seq.count("C")) / len(seq) for seq in sequences]).reshape(-1, 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(gc_content, labels, test_size=0.2, random_state=42)

# Train a random forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict the labels for the testing set
y_pred = clf.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.00


In [ ]:
from Bio import SeqIO
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

# Load the genome sequences in FASTA format
sequences = []
labels = []
for record in SeqIO.parse("coronavirus.fasta", "fasta"):
    sequences.append(str(record.seq))
    labels.append(record.id)

# Tokenize the sequences and pad them to a fixed length
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(sequences)
sequences = tokenizer.texts_to_sequences(sequences)
maxlen = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, padding='post', maxlen=maxlen)

# Convert the labels to one-hot encoding
num_classes = len(set(labels))
label_map = {label: i for i, label in enumerate(set(labels))}
labels = [label_map[label] for label in labels]
labels = np.eye(num_classes)[labels]

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.2, random_state=42)

# Define the LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model and train it on the training set
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model on the testing set
score, accuracy = model.evaluate(X_test, y_test, batch_size=32)
print(f"Accuracy: {accuracy:.2f}")


Epoch 1/10
1/1 [==============================] - 49s 49s/step - loss: 2.1992 - accuracy: 0.1429 - val_loss: 2.2539 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 39s 39s/step - loss: 2.1853 - accuracy: 0.2857 - val_loss: 2.2836 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 38s 38s/step - loss: 2.1714 - accuracy: 0.4286 - val_loss: 2.3153 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 40s 40s/step - loss: 2.1568 - accuracy: 0.2857 - val_loss: 2.3497 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 39s 39s/step - loss: 2.1497 - accuracy: 0.1429 - val_loss: 2.3887 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 38s 38s/step - loss: 2.1284 - accuracy: 0.4286 - val_loss: 2.4338 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 39s 39s/step - loss: 2.1148 - accuracy: 0.2857 - val_loss: 2.4878 - val_accuracy: 0.0000e+00

In [ ]:
from Bio import SeqIO
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC

# Load the genome sequences in FASTA format
sequences = []
labels = []
for record in SeqIO.parse("coronavirus.fasta", "fasta"):
    sequences.append(str(record.seq))
    labels.append(record.id)

# Vectorize the sequences based on k-mers
kmer_size = 4
vectorizer = CountVectorizer(analyzer='char', ngram_range=(kmer_size, kmer_size))
X = vectorizer.fit_transform(sequences).toarray()

# Convert the labels to integers
label_map = {label: i for i, label in enumerate(set(labels))}
y = np.array([label_map[label] for label in labels])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the SVM model and train it on the training set
model = SVC(kernel='linear')
model.fit(X_train, y_train)

# Evaluate the model on the testing set
accuracy = model.score(X_test, y_test)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.00
